STGCN trained on simulated streamflow.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets, utils, stgcn
import random
import pickle
import json
import networkx as nx

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190826-160713'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

CUDA Available
2019-08-26 16:07:13,915 - 20190826-160713 - cuda devices: ['Tesla V100-SXM2-16GB']


In [4]:
partitioning_strategy = 'distance'  # 'distance' or 'unilabel', see https://arxiv.org/abs/1801.07455
max_hops = 3 if partitioning_strategy == 'distance' else None
rdrs_vars = list(range(8))
agg = ['minmax','minmax','minmax','minmax','sum','minmax','minmax','minmax']
include_month = True
dem, landcover, soil, groundwater = False, False, False, False
landcover_types = []
seq_len = 14
seq_steps = 1

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'
spatial_val = True  # Whether the val set does spatial or temporal validation
val_fraction = 0.1

In [5]:
with open('../data/simulations_shervan/subbasins.geojson', 'r') as f:
     subbasin_shapes = json.loads(f.read())

subbasin_graph = utils.create_subbasin_graph()
component_graph = subbasin_graph.copy()
component_graph.remove_nodes_from(['sub-1', 'sub1', 'sub474'])  # remove Lake Erie and sink to get connected components
connected_components = sorted(sorted(c) for c in nx.connected_components(nx.Graph(component_graph)))

# Split into train/test/val regions
test_subbasins = [1]  # Do not include 474 (Lake Erie), because it has _very_ different properties (e.g. cell_count will be huge!)
train_subbasins = []
val_subbasins = []
train_components = []
np.random.seed(1)
for component in connected_components:
    max_y = -999
    for node in component:
        subbasin = list(s['properties'] for s in subbasin_shapes['features'] if 'sub' + str(s['properties']['SubId']) == node)[0]
        max_y = max(max_y, subbasin['INSIDE_Y'])
    if max_y < 42.8:
        if spatial_val and np.random.choice([True, False], size=1, p=[val_fraction, 1-val_fraction]):
            val_subbasins += list(int(c[3:]) for c in component)
        else:
            train_components.append(list(int(c[3:]) for c in component))
            train_subbasins += train_components[-1]
    else:
        test_subbasins += list(int(c[3:]) for c in component)
        
if not spatial_val: # if no spatial validation, use same graph but different samples
    val_subbasins = train_subbasins
train_subbasins, val_subbasins, test_subbasins = sorted(train_subbasins), sorted(val_subbasins), sorted(test_subbasins)

# For each train-component, get its nodes' indices into train_subbasins. 
# We'll use them to create the adjacency matrix of the component during training.
train_component_indices = []
for component in train_components:
    train_component_indices.append(sorted(train_subbasins.index(c) for c in component))

train_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in train_subbasins))
val_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in val_subbasins))
test_subgraph = subbasin_graph.subgraph(list('sub' + str(t) for t in test_subbasins))
if partitioning_strategy == 'unilabel':
    train_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(train_subgraph, nodelist=list('sub' + str(t) for t in train_subbasins))), 0).float().to(device)
    val_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(val_subgraph, nodelist=list('sub' + str(t) for t in val_subbasins))), 0).float().to(device)
    test_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(test_subraph, nodelist=list('sub' + str(t) for t in test_subbasins))), 0).float().to(device)
elif partitioning_strategy == 'distance':  # use distances in upstream-graph, i.e. in reversed downstream-graph
    train_adjacency = utils.create_hop_matrix(train_subgraph.reverse(), max_hops, list('sub' + str(t) for t in train_subbasins)).float().to(device)
    val_adjacency = utils.create_hop_matrix(val_subgraph.reverse(), max_hops, list('sub' + str(t) for t in val_subbasins)).float().to(device)
    test_adjacency = utils.create_hop_matrix(test_subgraph.reverse(), max_hops, list('sub' + str(t) for t in test_subbasins)).float().to(device)
else:
    raise Exception('Unsupported partitioning strategy')

subbasins = sorted(set(train_subbasins + test_subbasins + val_subbasins))

In [6]:
train_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                   dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
if spatial_val:
    val_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, val_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                     conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
else:
    val_dataset = train_dataset

# Two test datasets: one with spatial and temporal validation (i.e., different graph, different time), and one with only temporal validation (i.e. different time period only)
spatial_test_dataset = None
if test_subbasins != train_subbasins:
    spatial_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, test_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                              conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
temporal_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                           conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)

station_subbasins = train_dataset.simulated_streamflow[~pd.isna(train_dataset.simulated_streamflow['StationID'])]['subbasin'].unique()

Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2010-01-15000000-2012-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_519f4faef33ba86b89ab71774d2c9238.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2010-01-15000000-2012-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_2ea097a09daf62577b8793ad1f423cca.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2013-01-01-2014-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_4812ff52fd3d443cf075b68b4742a4d7.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_14-1_2013-01-01-2014-12-31___minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_519f4faef33ba86b89ab71774d2c9238.pkl


In [7]:
# Train model
num_epochs = 500
learning_rate = 2e-3
patience = 500
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)
dropout = 0.3
weight_decay = 1e-5

batch_size = 16
temp_kernel_size = 3
model = stgcn.Model(train_dataset.x.shape[2], train_adjacency.shape[0], temp_kernel_size, dropout=dropout).to(device)
if num_devices > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter(comment='STGCN_simulationTraining')
param_description = {'time_stamp': time_stamp, 'batch_size': batch_size, 'loss': loss_fn, 'include_month': include_month, 'aggregate_daily': agg, 'rdrs_vars': rdrs_vars, 'dropout': dropout, 'spatial_validation': spatial_val, 'val_fraction': val_fraction, 'temp_kernel': temp_kernel_size,
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'x_train_shape': train_dataset.x.shape, 'x_val_shape': val_dataset.x.shape, 'partitioning_strategy': partitioning_strategy, 'max_hops': max_hops,
                     'spatial_x_test_shape': spatial_test_dataset.x.shape if spatial_test_dataset is not None else '', 'temporal_x_test_shape': temporal_test_dataset.x.shape, 'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 
                     'landcover_types': landcover_types, 'test_start': test_start, 'test_end': test_end, 'model': str(model).replace('\n','').replace(' ', ''),}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190826-160713', 'batch_size': 16, 'loss': NSELoss(), 'include_month': True, 'aggregate_daily': ['minmax', 'minmax', 'minmax', 'minmax', 'sum', 'minmax', 'minmax', 'minmax'], 'rdrs_vars': [0, 1, 2, 3, 4, 5, 6, 7], 'dropout': 0.3, 'spatial_validation': True, 'val_fraction': 0.1, 'temp_kernel': 3, 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-05\n), 'lr': 0.002, 'patience': 500, 'min_improvement': 0.01, 'x_train_shape': torch.Size([1082, 14, 82, 383]), 'x_val_shape': torch.Size([1082, 14, 82, 56]), 'partitioning_strategy': 'distance', 'max_hops': 3, 'spatial_x_test_shape': torch.Size([730, 14, 82, 284]), 'temporal_x_test_shape': torch.Size([730, 14, 82, 383]), 'num_epochs': 500, 'seq_len': 14, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 15, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-05, 'landcover_types': [], 'test_start': '2013-01-01', 'test_end': '2014-12

In [8]:
if spatial_val:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
else:
    val_indices = np.random.choice(len(train_dataset), size=int(val_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    
if spatial_test_dataset is not None:
    spatial_test_dataloader = torch.utils.data.DataLoader(spatial_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
temporal_test_dataloader = torch.utils.data.DataLoader(temporal_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [9]:
# Subbasins with constant streamflow will divide by zero in loss calculation. Calculate loss without them.
train_non_constant_subbasin_mask = ((train_dataset.y_sim.min(dim=0)[0] - train_dataset.y_sim.max(dim=0)[0]) != 0).to(device)
val_non_constant_subbasin_mask = ((val_dataset.y_sim.min(dim=0)[0] - val_dataset.y_sim.max(dim=0)[0]) != 0).to(device)

y_train_means = train_dataset.y_sim_means.to(device)
y_val_means = val_dataset.y_sim_means[val_non_constant_subbasin_mask].to(device)

In [10]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    train_losses = torch.tensor(0.0)
    for i, train_batch in enumerate(train_dataloader):
        # in each batch, train on a random subset of connected component to foster generalization
        component_idxs = sorted(n for comp in np.random.choice(train_component_indices, size=len(train_component_indices)//3, replace=False) for n in comp)
        mask = train_non_constant_subbasin_mask[component_idxs]
        adj = train_adjacency[:,component_idxs][:,:,component_idxs]
        y_pred = model(train_batch['x'].permute(0,2,1,3)[...,component_idxs].to(device), adj)
        train_loss = loss_fn(y_pred[:,mask], train_batch['y_sim'][:,component_idxs][:,mask].to(device), means=y_train_means[component_idxs][mask])
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_losses += train_loss.detach()
        
    train_loss = (train_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(train_loss))
    writer.add_scalar('loss_nse', train_loss, epoch)
    
    model.eval()
    val_losses = torch.tensor(0.0)
    for i, val_batch in enumerate(val_dataloader):
        y_pred = model(val_batch['x'].permute(0,2,1,3).to(device), val_adjacency).detach()
        val_losses += loss_fn(y_pred[:,val_non_constant_subbasin_mask], val_batch['y_sim'][:,val_non_constant_subbasin_mask].to(device), means=y_val_means).detach()
        
    val_loss = (val_losses / len(val_dataloader)).item()
    print('Epoch', epoch, 'mean val loss:\t{}'.format(val_loss))
    writer.add_scalar('loss_nse_val', val_loss, epoch)
    
    if val_loss < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
        load_data.pickle_model('STGCN_simulationTraining', model, 'allStations', time_stamp, model_type='torch.dill')
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break
    
print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.save_model_with_state('STGCN_simulationTraining', best_loss_model[0], model, optimizer, time_stamp, use_dill=True)

Epoch 0 mean train loss:	5.1228437423706055
Epoch 0 mean val loss:	5.352047920227051
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190826-160713.pkl
Epoch 1 mean train loss:	1.3391093015670776
Epoch 1 mean val loss:	4.250066757202148
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190826-160713.pkl
Epoch 2 mean train loss:	1.2851628065109253
Epoch 2 mean val loss:	5.127552032470703
Epoch 3 mean train loss:	1.2242317199707031
Epoch 3 mean val loss:	2.905252695083618
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190826-160713.pkl
Epoch 4 mean train loss:	1.1337405443191528
Epoch 4 mean val loss:	2.1644270420074463
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190826-160713.pkl
Epoch 5 mean train loss:	1.0740810632705688
Epoch 5 mean val loss:	1.1475419998168945
Saved model as /home/mga

In [11]:
del y_train_means, y_val_means, y_pred, train_non_constant_subbasin_mask, val_non_constant_subbasin_mask
if USE_CUDA:
    torch.cuda.empty_cache()

In [12]:
logger.warning('predicting')
model.eval()

temporal_test_predictions = []  # test on train graph but different time
for i, test_batch in enumerate(temporal_test_dataloader):
    pred = model(test_batch['x'].permute(0,2,1,3).to(device), train_adjacency).detach().cpu()
    temporal_test_predictions.append(pred)
predictions = torch.cat(temporal_test_predictions)

if spatial_test_dataset is not None:
    spatial_test_predictions = []  # test on different graph, different time
    for i, test_batch in enumerate(spatial_test_dataloader):
        pred = model(test_batch['x'].permute(0,2,1,3).to(device), test_adjacency).detach().cpu()
        spatial_test_predictions.append(pred)
    predictions = torch.cat([torch.cat(spatial_test_predictions), predictions], dim=1)

2019-08-26 16:54:52,081 - 20190826-160713 - predicting


In [13]:
actuals = temporal_test_dataset.data_streamflow.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_dict, nse_sim_dict = {}, {}
mse_dict, mse_sim_dict = {}, {}
predictions_df = pd.DataFrame(columns=actuals.columns)
predictions_df['is_test_subbasin'] = False
all_test_subbasins = test_subbasins + train_subbasins if train_subbasins != test_subbasins else train_subbasins
for i in range(len(all_test_subbasins)):
    subbasin = all_test_subbasins[i]
    station = None
    subbasin_sim = temporal_test_dataset.simulated_streamflow[temporal_test_dataset.simulated_streamflow['subbasin'] == subbasin].set_index('date')
    if subbasin in station_subbasins:
        station = subbasin_sim['StationID'].values[0]
        act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != subbasin_sim.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for subbasin {}. Ignoring excess actuals.'.format(len(predictions), len(subbasin_sim), subbasin))
        subbasin_sim = subbasin_sim.iloc[:predictions.shape[0]]
        if station is not None:
            act = act.iloc[:predictions.shape[0]]
            
    pred = pd.DataFrame({'runoff': predictions[:,i]}, index=subbasin_sim.index)
    pred['subbasin'] = subbasin
    pred['station'] = station
    pred['is_test_subbasin'] = subbasin in test_subbasins
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    subbasin_type = 'test' if subbasin in test_subbasins else 'train'
    nse_sim, mse_sim = evaluate.evaluate_daily('Sub{}'.format(subbasin), pred['runoff'], subbasin_sim['simulated_streamflow'], writer=writer, group=subbasin_type)
    nse_sim_dict[subbasin] = nse_sim
    mse_sim_dict[subbasin] = mse_sim

    if station is not None:
        nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
        nse_dict[subbasin] = nse
        mse_dict[subbasin] = mse
        print(station, subbasin, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')
    print(subbasin, '\tNSE sim:', nse_sim, '\tMSE sim:', mse_sim)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


1 	NSE sim: -1.174287578811382 	MSE sim: 2515494.0319927847
2 	NSE sim: -0.9848549961563122 	MSE sim: 168.4695042903008
3 	NSE sim: -1.0780833560691323 	MSE sim: 66.12868093772595
4 	NSE sim: -0.1390004525698907 	MSE sim: 19.40801563327929
5 	NSE sim: -0.06106667217845585 	MSE sim: 25.735592557848168
8 	NSE sim: -2.2831565650298224 	MSE sim: 24.993004008402323
9 	NSE sim: -0.07724198253014003 	MSE sim: 4.372656515791979
10 	NSE sim: -0.17143748474098675 	MSE sim: 6.677000666422932
11 	NSE sim: 0.49803071959403533 	MSE sim: 6.125602471522579
12 	NSE sim: 0.11837164636199715 	MSE sim: 2.5944517555755198
18 	NSE sim: -0.9363159740469693 	MSE sim: 13919.154145713783
21 	NSE sim: -0.9282101835534382 	MSE sim: 13583.21273122735
25 	NSE sim: -0.8673391969633506 	MSE sim: 11391.491826829402
26 	NSE sim: -0.2718419170947317 	MSE sim: 37.12638736692563
27 	NSE sim: 0.4175539471943419 	MSE sim: 3.6348004712381385
28 	NSE sim: 0.39651662327970805 	MSE sim: 4.621164474554851
29 	NSE sim: 0.44759606

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


502 	NSE sim: 0.6166930035087453 	MSE sim: 0.03996073084394657
503 	NSE sim: 0.6155076145274243 	MSE sim: 0.010376308823671571
505 	NSE sim: 0.5441778476750936 	MSE sim: 1.2962061946085968
506 	NSE sim: 0.6142184129240325 	MSE sim: 13.565692382844942
507 	NSE sim: 0.5491677584243633 	MSE sim: 0.006279509472262113
508 	NSE sim: 0.625980589634793 	MSE sim: 0.9747065547726993
509 	NSE sim: 0.5696006890031652 	MSE sim: 24.766730977353802
511 	NSE sim: 0.533974587535059 	MSE sim: 0.33945352939949397
512 	NSE sim: 0.5643670924891453 	MSE sim: 102.11103795441328
513 	NSE sim: 0.5570926723908824 	MSE sim: 97.30572168300486
514 	NSE sim: 0.4936648383280232 	MSE sim: 0.4802206898259382
515 	NSE sim: 0.47268887269619553 	MSE sim: 0.10349507294890306
516 	NSE sim: 0.5459511577513452 	MSE sim: 0.1749324402840582
517 	NSE sim: 0.4871441644423882 	MSE sim: 0.01647160328076355
518 	NSE sim: 0.5458465500037271 	MSE sim: 0.004141967818310679
521 	NSE sim: -0.2459309681040538 	MSE sim: 5.447166974712119


/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


525 	NSE sim: nan 	MSE sim: 0.11500641040616254
527 	NSE sim: 0.5463289550707373 	MSE sim: 0.007438680205375195
528 	NSE sim: 0.5399462160934798 	MSE sim: 0.031073246556715835
529 	NSE sim: 0.5268339785074365 	MSE sim: 0.0048772372497400285
530 	NSE sim: 0.36383709873668035 	MSE sim: 3.35743521268233
531 	NSE sim: -0.1357443705123904 	MSE sim: 1.3575043691596838
532 	NSE sim: -0.09779067664574992 	MSE sim: 0.36082992958866517
533 	NSE sim: 0.37263320853543447 	MSE sim: 0.0012324721724622412
534 	NSE sim: 0.5371693286683279 	MSE sim: 0.2829616146162023
535 	NSE sim: 0.36281887110860433 	MSE sim: 0.19804446170498577
536 	NSE sim: 0.3970403688286839 	MSE sim: 1.6153106068383707
537 	NSE sim: 0.39654974314401814 	MSE sim: 0.9745569221336612
538 	NSE sim: -0.4597187240433853 	MSE sim: 249.8808942733941
539 	NSE sim: 0.3815844298505875 	MSE sim: 0.01201890547683839
540 	NSE sim: 0.48163667733341964 	MSE sim: 2702.9654046785136
541 	NSE sim: 0.5935715690227987 	MSE sim: 219.49025458984227
542

In [14]:
def print_nse_mse(name, nse_dict, mse_dict, subbasins):
    nses = list(nse_dict[s] for s in subbasins)
    mses = list(mse_dict[s] for s in subbasins)
    print(name, 'Median NSE (clipped to 0)', np.nanmedian(nses), '/ Min', np.nanmin(nses), '/ Max', np.nanmax(nses))
    print(' ' * len(name), 'Median MSE (clipped to 0)', np.nanmedian(mses), '/ Min', np.nanmin(mses), '/ Max', np.nanmax(mses))
    
    return np.nanmedian(nses)

nse_median_sim_temporal = print_nse_mse('Temporal test sim', nse_sim_dict, mse_sim_dict, train_subbasins)
nse_median_stations_temporal = print_nse_mse('Stations temporal test', nse_dict, mse_dict, list(s for s in station_subbasins if s in train_subbasins))
if spatial_test_dataset is not None:
    nse_median_sim_spatial = print_nse_mse('Spatial test sim', nse_sim_dict, mse_sim_dict, test_subbasins)
    nse_median_stations_spatial = print_nse_mse('Stations spatial test', nse_dict, mse_dict, list(s for s in station_subbasins if s in test_subbasins))
    writer.add_scalar('nse_median_sim', nse_median_sim_spatial)
    writer.add_scalar('nse_median_stations_spatial', nse_median_stations_spatial)
writer.add_scalar('nse_median_sim_temporal', nse_median_sim_temporal)
writer.add_scalar('nse_median_stations_temporal', nse_median_stations_temporal)

Temporal test sim Median NSE (clipped to 0) 0.4446940233798964 / Min -1.4459407382651697 / Max 0.6490510191355752
                  Median MSE (clipped to 0) 5.531612549992785 / Min 0.0012324721724622412 / Max 56364.30971085532
Stations temporal test Median NSE (clipped to 0) 0.1340020771192867 / Min -1.5525962196881524 / Max 0.3331320453781951
                       Median MSE (clipped to 0) 470.4869427373762 / Min 21.60376211941555 / Max 49868.03789385802
Spatial test sim Median NSE (clipped to 0) -0.6615612526486421 / Min -752749.9695560089 / Max 0.5743971186819846
                 Median MSE (clipped to 0) 227.16996227103212 / Min 0.009204064626416944 / Max 2515494.0319927847
Stations spatial test Median NSE (clipped to 0) -0.7539960578111904 / Min -3365.2357509642884 / Max 0.07884824338939134
                      Median MSE (clipped to 0) 318.66613424240853 / Min 6.85705147873416 / Max 141469.5161740062


In [15]:
nse_dict

{676: -0.2249922392376391,
 677: -1.24502578035155,
 678: -15.961193704743224,
 680: -1561.819955763022,
 681: -3365.2357509642884,
 682: -1358.0624524778318,
 683: -0.0625001024377676,
 684: -0.2688118605761509,
 685: -0.46424007149982827,
 686: 0.00984948860117385,
 687: -0.5564519602383509,
 688: -0.033783280112957836,
 689: 0.06494425541280613,
 690: -0.4991336099548169,
 691: -3190.0948834250553,
 692: -47.16078926872062,
 693: -806.3483391524717,
 694: -723.6459601800769,
 695: -3164.9802620274236,
 696: -174.59230978208203,
 697: -2740.767988550998,
 698: -10.874104512251574,
 699: 0.07884824338939134,
 700: -0.18279854698299025,
 701: -0.04080608159245136,
 702: -0.7539960578111904,
 703: -1.6726578176139948,
 705: -0.10428944421099962,
 706: -0.141608173398305,
 707: -0.07723270737469656,
 709: 0.3331320453781951,
 710: 0.2182121395805099,
 712: 0.16849784561665482,
 713: 0.295656636270772,
 714: 0.16186949783706128,
 716: 0.1340020771192867,
 717: -0.016785991925436283,
 718:

In [16]:
writer.close()

In [17]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), 
                   temporal_test_dataset.simulated_streamflow, on=['date', 'subbasin'])
save_df = pd.merge(save_df, actuals.rename({'runoff': 'actual'}, axis=1), how='left', on=['date', 'station'])\
            [['date', 'subbasin', 'station', 'prediction', 'actual', 'simulated_streamflow', 'is_test_subbasin']]
load_data.pickle_results('STGCN_simulationTraining', save_df, time_stamp)

'STGCN_simulationTraining_20190826-160713.pkl'

In [18]:
_ = print(train_subbasins), print(val_subbasins), print(test_subbasins)

[6, 7, 41, 45, 51, 57, 60, 62, 69, 70, 82, 85, 88, 89, 90, 91, 96, 97, 98, 99, 101, 102, 103, 104, 106, 107, 108, 109, 112, 113, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 155, 156, 158, 159, 160, 163, 164, 165, 166, 169, 170, 171, 172, 173, 178, 179, 180, 181, 182, 183, 186, 187, 188, 189, 191, 194, 195, 198, 199, 200, 205, 207, 210, 212, 213, 214, 215, 216, 217, 218, 221, 223, 224, 225, 226, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 240, 241, 242, 243, 244, 245, 253, 254, 255, 256, 257, 266, 267, 272, 273, 274, 279, 280, 296, 297, 298, 299, 300, 301, 306, 308, 310, 311, 313, 316, 317, 318, 319, 320, 321, 322, 327, 328, 329, 330, 331, 332, 334, 336, 339, 340, 341, 342, 343, 344, 347, 348, 349, 352, 353, 355, 356, 357, 358, 359, 361, 362, 363, 364, 365, 366, 371, 372, 373, 374, 375, 376, 377, 380, 381, 382, 386, 387, 390, 393, 394, 395, 3

In [19]:
list(k for k in list(nse_dict.keys()) if k in test_subbasins)

[676,
 677,
 678,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 705,
 706]

In [20]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190826-165630'